In [1]:
import pandas as pd

In [2]:
korean_drama = pd.read_csv('./korean_drama_title_real.csv')

In [ ]:
korean_drama

In [ ]:
korean_drama.groupby('year').count()

In [ ]:
korean_drama.groupby('tot_eps').count()

In [ ]:
korean_drama[korean_drama['tot_eps']==5]

In [ ]:
korean_drama.isnull()

In [ ]:
korean_drama.sort_values(by='rank')

In [3]:
korean_drama

,kdrama_id,title,year,director,screenwriter,country,type,tot_eps,duration,start_dt,end_dt,aired_on,org_net,content_rt,synopsis,rank,pop,korean_title
0,661d4193916c4e71a2c70473ab11e9e8,Sing My Crush,2023,['So Joon Moon'],NaN,South Korea,Drama,8,1500.0,"Aug 2, 2023","Aug 2, 2023",Wednesday,NaN,Not Yet Rated,Follow the story of acquaintances Ba Ram and H...,1484,2238,따라바람
1,5ffcbeaa17114714af1959129984274c,D.P. Season 2,2023,NaN,['Kim Bo Tong'],South Korea,Drama,6,3000.0,"Jul 28, 2023","Jul 28, 2023",Friday,Netflix,15+ - Teens 15 or older,This unfolding story ensues when military dese...,164,1084,디피 시즌2
2,65075cb9c1a54be4a441cee6f16c9fdf,Shadow Detective Season 2,2023,['Han Dong Hwa'],"['Song Jung Woo', 'Hwang Seol Hun']",South Korea,Drama,8,3300.0,2023-07-05,2023-07-26,Wednesday,Disney+ Hulu,15+ - Teens 15 or older,Unfolds the ultimate counterattack of veteran ...,2443,6915,형사록 시즌2
3,df0f0ac4b3ff4b15afa26f5a7a53a328,To Be Honest,2023,NaN,NaN,South Korea,Drama,3,600.0,2023-06-30,2023-07-14,Friday,NaN,Not Yet Rated,Don't you have those days where the whole univ...,49895,99999,솔직하게 말해서
4,04c1fe41948e464fb440001831d74d41,Celebrity,2023,['Kim Chul Gyu'],['Kim Yi Young'],South Korea,Drama,12,2700.0,"Jun 30, 2023","Jun 30, 2023",Friday,Netflix,18+ Restricted (violence & profanity),Fame. Money. Power. One young woman fights to ...,826,547,셀러브리티
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,8a720bdcf2234bda87d53eb50736f699,Winter Vacation,2020,NaN,['Kim Hyo Jin'],South Korea,Drama,6,660.0,2020-01-10,2020-02-14,Friday,vLive,15+ - Teens 15 or older,It is a teen romance web drama that tells the ...,58926,13663,겨울방학
772,75b8778871df4fbc830652abeb13fa89,A Twist Girl,2020,NaN,NaN,South Korea,Drama,3,600.0,2020-01-09,2020-02-08,NaN,NaN,Not Yet Rated,The first-class girl in the whole school and a...,52785,99999,반전소녀
773,79c4732493654aea9f181811e289adc4,Dr. Romantic Season 2,2020,"['Yoo In Shik', 'Lee Gil Bok']",['Kang Eun Kyung'],South Korea,Drama,16,4200.0,2020-01-06,2020-02-25,"Monday, Tuesday",SBS,15+ - Teens 15 or older,"A “real doctor” story set in a small, humble h...",185,168,낭만닥터 김사부2
774,544c51af0c98459bb4e3ef922b56fffd,Lover of the Palace,2020,NaN,NaN,South Korea,Drama,10,600.0,2020-01-03,2020-01-17,NaN,NaN,Not Yet Rated,"Working as a tour guide at Deoksugung Palace, ...",8535,8135,궁궐의연인


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [ ]:
# 웹 드라이버 초기화
driver = webdriver.Chrome()

drama_eng_title = korean_drama['title']

def drama_genre(x):
    global drama_eng_title
    name = x
    url = f"https://mydramalist.com/search?q={name.replace(' ', '+')}&adv=titles&ty=68,83&so=relevance"
    driver.get(url)
    try:
        dramas = driver.find_element(By.XPATH, "//*[@id]/div/div/div[2]/h6/a[1]").send_keys(Keys.ENTER)

        drama_genre = WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='show-detailsxx']/div[4]/ul[1]/li[4]")))
        drama_genre_text = drama_genre.text
        if 'Genres' in drama_genre_text:
            drama_genre_text = drama_genre.text
            
        elif 'Tags' in drama_genre_text:
            drama_genre = WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='show-detailsxx']/div[4]/ul[1]/li[5]")))
            drama_genre_text = drama_genre.text  
        elif 'Screenwriter' in drama_genre_text:
            drama_genre = WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='show-detailsxx']/div[4]/ul[1]/li[6]")))
            drama_genre_text = drama_genre.text     
        else: 
            drama_genre = WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='show-detailsxx']/div[4]/ul[1]/li[3]")))
            drama_genre_text = drama_genre.text       
    except TimeoutException as ex:
        drama_genre_text = 'Notfound'
    return drama_genre_text
# 결과를 저장할 빈 리스트 생성
drama_genres = []

i=0
# 영문 제목 번역 
for title in drama_eng_title:
    time.sleep(3)
    genre = drama_genre(title)
    drama_genres.append(genre)
    print(i, title, genre)
    i+=1
# 장르 결과
korean_drama['genre'] = drama_genres

driver.quit()

In [5]:
# 웹 드라이버 초기화
driver = webdriver.Chrome()

drama_eng_title = korean_drama['title']

def drama_genre(x):
    global drama_eng_title
    name = x
    url = f"https://mydramalist.com/search?q={name.replace(' ', '+')}&adv=titles&ty=68,83&so=relevance"
    driver.get(url)
    
    dramas = driver.find_element(By.XPATH, "//*[@id]/div/div/div[2]/h6/a[1]").send_keys(Keys.ENTER)
    try:
        drama_genre = driver.find_element(By.CLASS_NAME, "show-genres")
        drama_genre_text = drama_genre.text
    except TimeoutException as ex:
        drama_genre_text = "Notfound"
    return drama_genre_text

drama_genres = []

i=0
# 영문 제목 번역 
for title in drama_eng_title:
    time.sleep(3)
    genre = drama_genre(title)
    drama_genres.append(genre)
    print(i, title, genre)
    i+=1
# 장르 결과
korean_drama['genre'] = drama_genres

driver.quit()

0 Sing My Crush Genres: Music, Comedy, Romance, Youth
1 D.P. Season 2 Genres: Action, Military, Drama
2 Shadow Detective Season 2 Genres: Mystery, Drama
3 To Be Honest Genres: Youth, Drama
4 Celebrity Genres: Thriller, Mystery, Drama
5 Blue Temperature Genres: Romance, Youth
6 Numbers Genres: Action, Thriller, Mystery, Drama
7 Revenant Genres: Thriller, Mystery, Horror, Supernatural
8 Adult Kids Genres: Life, Drama
9 Lies Hidden in My Garden Genres: Thriller, Mystery, Psychological, Drama
10 King the Land Genres: Comedy, Romance
11 See You in My 19th Life Genres: Comedy, Romance, Drama, Fantasy
12 Bloodhounds Genres: Action, Thriller, Crime, Drama
13 Love Tractor Genres: Comedy, Romance, Life
14 Romance by Romance Genres: Romance
15 The Villain of Romance Genres: Romance
16 Sound Candy Genres: Drama
17 Bitch X Rich Genres: Thriller, Mystery, Youth, Drama
18 Battle for Happiness Genres: Thriller, Mystery, Drama
19 Delightfully Deceitful Genres: Comedy, Crime, Drama
20 One Day Off Genres

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".show-genres"}
  (Session info: chrome=116.0.5845.180); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF65D3A52A2+57122]
	(No symbol) [0x00007FF65D31EA92]
	(No symbol) [0x00007FF65D1EE3AB]
	(No symbol) [0x00007FF65D227D3E]
	(No symbol) [0x00007FF65D227E2C]
	(No symbol) [0x00007FF65D260B67]
	(No symbol) [0x00007FF65D24701F]
	(No symbol) [0x00007FF65D25EB82]
	(No symbol) [0x00007FF65D246DB3]
	(No symbol) [0x00007FF65D21D2B1]
	(No symbol) [0x00007FF65D21E494]
	GetHandleVerifier [0x00007FF65D64EF82+2849794]
	GetHandleVerifier [0x00007FF65D6A1D24+3189156]
	GetHandleVerifier [0x00007FF65D69ACAF+3160367]
	GetHandleVerifier [0x00007FF65D436D06+653702]
	(No symbol) [0x00007FF65D32A208]
	(No symbol) [0x00007FF65D3262C4]
	(No symbol) [0x00007FF65D326

In [6]:
print(drama_genres)

['Genres: Music, Comedy, Romance, Youth', 'Genres: Action, Military, Drama', 'Genres: Mystery, Drama', 'Genres: Youth, Drama', 'Genres: Thriller, Mystery, Drama', 'Genres: Romance, Youth', 'Genres: Action, Thriller, Mystery, Drama', 'Genres: Thriller, Mystery, Horror, Supernatural', 'Genres: Life, Drama', 'Genres: Thriller, Mystery, Psychological, Drama', 'Genres: Comedy, Romance', 'Genres: Comedy, Romance, Drama, Fantasy', 'Genres: Action, Thriller, Crime, Drama', 'Genres: Comedy, Romance, Life', 'Genres: Romance', 'Genres: Romance', 'Genres: Drama', 'Genres: Thriller, Mystery, Youth, Drama', 'Genres: Thriller, Mystery, Drama', 'Genres: Comedy, Crime, Drama', 'Genres: Food, Life, Drama', 'Genres: Romance, Youth', 'Genres: Comedy, Romance', 'Genres: Action, Adventure, Drama, Sci-Fi', 'Genres: Business, Romance, Drama, Melodrama', 'Genres: Comedy, Romance, Youth, Drama', 'Genres: Historical, Horror, Romance, Fantasy', 'Genres: Romance, Youth', 'Genres: Comedy, Romance, Drama', 'Genres: 

In [11]:
for i in range(109):
    korean_drama.loc[i, 'genre'] = drama_genres[i]


In [14]:
korean_drama.loc[109]

kdrama_id       c123bdee96414ea88041f11b0f046000
title                      Why Opposites Attract
year                                        2022
director                                     NaN
screenwriter                                 NaN
country                              South Korea
type                                       Drama
tot_eps                                        8
duration                                   480.0
start_dt                              2022-11-21
end_dt                                2023-01-02
aired_on                                     NaN
org_net                                      NaN
content_rt                         Not Yet Rated
synopsis                                     NaN
rank                                       57166
pop                                        99999
korean_title                          반대가 끌리는 이유
genre                                        NaN
Name: 109, dtype: object

In [20]:
driver = webdriver.Chrome()

drama_eng_title = korean_drama.loc[109:776]['title']

def drama_genre(x):
    global drama_eng_title
    name = x
    url = f"https://mydramalist.com/search?q={name.replace(' ', '+')}&adv=titles&ty=68,83&so=relevance"
    driver.get(url)
    
    dramas = driver.find_element(By.XPATH, "//*[@id]/div/div/div[2]/h6/a[1]").send_keys(Keys.ENTER)
    try:
        drama_genre = driver.find_element(By.CLASS_NAME, "show-genres")
        drama_genre_text = drama_genre.text
    except Exception as ex:
        drama_genre_text = "Notfound"
    return drama_genre_text

drama_genres = []

i=0
# 영문 제목 번역 
for title in drama_eng_title:
    time.sleep(3)
    genre = drama_genre(title)
    drama_genres.append(genre)
    print(i, title, genre)
    i+=1
# 장르 결과
korean_drama['genre'] = drama_genres

driver.quit()

0 Why Opposites Attract Notfound
1 Out of this World Girlfriend Genres: Thriller, Comedy, Youth, Sci-Fi
2 Summer Strike Genres: Romance, Life, Drama, Melodrama
3 New Love Playlist Genres: Romance, Youth, Drama
4 Fanletter, Please Genres: Comedy, Romance
5 Weak Hero Class 1 Genres: Action, Youth, Drama
6 Reborn Rich Genres: Business, Drama, Fantasy, Political
7 Somebody Genres: Thriller, Mystery, Psychological, Mature
8 Phantom School Genres: Action, Youth, Fantasy
9 The First Responders Genres: Action, Thriller, Mystery, Drama
10 Revenge of Others Genres: Action, Thriller, Mystery, Drama
11 Chart Running Scholar Genres: Romance, Fantasy
12 The Witch Store Reopening Genres: Comedy, Romance, Youth, Fantasy
13 Behind Every Star Genres: Comedy, Romance, Life
14 Disarming Romance Genres: Comedy, Romance, Youth, Drama
15 Our Relationship Ended Before It Began Genres: Romance, Drama
16 Curtain Call Genres: Romance, Life, Drama, Family
17 Idol in My Living Room Genres: Romance, Youth, Melodram

TimeoutException: Message: timeout: Timed out receiving message from renderer: 299.186
  (Session info: chrome=116.0.5845.180)
Stacktrace:
	GetHandleVerifier [0x00007FF65D3A52A2+57122]
	(No symbol) [0x00007FF65D31EA92]
	(No symbol) [0x00007FF65D1EE3AB]
	(No symbol) [0x00007FF65D1DC01A]
	(No symbol) [0x00007FF65D1DBF63]
	(No symbol) [0x00007FF65D1DA9E5]
	(No symbol) [0x00007FF65D1DB31E]
	(No symbol) [0x00007FF65D1E80CB]
	(No symbol) [0x00007FF65D1F7F31]
	(No symbol) [0x00007FF65D1FC0DA]
	(No symbol) [0x00007FF65D1DB8E7]
	(No symbol) [0x00007FF65D1F7E0A]
	(No symbol) [0x00007FF65D25EBE9]
	(No symbol) [0x00007FF65D246DB3]
	(No symbol) [0x00007FF65D21D2B1]
	(No symbol) [0x00007FF65D21E494]
	GetHandleVerifier [0x00007FF65D64EF82+2849794]
	GetHandleVerifier [0x00007FF65D6A1D24+3189156]
	GetHandleVerifier [0x00007FF65D69ACAF+3160367]
	GetHandleVerifier [0x00007FF65D436D06+653702]
	(No symbol) [0x00007FF65D32A208]
	(No symbol) [0x00007FF65D3262C4]
	(No symbol) [0x00007FF65D3263F6]
	(No symbol)

In [21]:
drama_genres

['Notfound',
 'Genres: Thriller, Comedy, Youth, Sci-Fi',
 'Genres: Romance, Life, Drama, Melodrama',
 'Genres: Romance, Youth, Drama',
 'Genres: Comedy, Romance',
 'Genres: Action, Youth, Drama',
 'Genres: Business, Drama, Fantasy, Political',
 'Genres: Thriller, Mystery, Psychological, Mature',
 'Genres: Action, Youth, Fantasy',
 'Genres: Action, Thriller, Mystery, Drama',
 'Genres: Action, Thriller, Mystery, Drama',
 'Genres: Romance, Fantasy',
 'Genres: Comedy, Romance, Youth, Fantasy',
 'Genres: Comedy, Romance, Life',
 'Genres: Comedy, Romance, Youth, Drama',
 'Genres: Romance, Drama',
 'Genres: Romance, Life, Drama, Family',
 'Genres: Romance, Youth, Melodrama',
 'Genres: Thriller, Comedy, Crime, Drama',
 'Genres: Mystery, Horror, Life, Supernatural',
 'Genres: Mystery, Crime, Drama',
 'Genres: Comedy, Romance, Drama, Fantasy',
 'Genres: Comedy, Romance',
 'Genres: Romance, Youth, Fantasy',
 'Genres: Mystery, Romance, Drama, Supernatural',
 'Genres: Music, Romance',
 'Genres: His

In [26]:
len(drama_genres)

330

In [23]:
korean_drama[korean_drama['title']=='The Devil Judge']

,kdrama_id,title,year,director,screenwriter,country,type,tot_eps,duration,start_dt,end_dt,aired_on,org_net,content_rt,synopsis,rank,pop,korean_title,genre
438,22dca69dbdb74efa9566fa28551fbf5e,The Devil Judge,2021,['Choi Jung Gyu'],['Moon Yoo Seok'],South Korea,Drama,16,4500.0,2021-07-03,2021-08-22,"Saturday, Sunday",tvN,15+ - Teens 15 or older,Set in a dystopian version of present-day Kore...,170,165,악마판사,NaN


In [25]:
korean_drama.loc[109:438]

,kdrama_id,title,year,director,screenwriter,country,type,tot_eps,duration,start_dt,end_dt,aired_on,org_net,content_rt,synopsis,rank,pop,korean_title,genre
109,c123bdee96414ea88041f11b0f046000,Why Opposites Attract,2022,NaN,NaN,South Korea,Drama,8,480.0,2022-11-21,2023-01-02,NaN,NaN,Not Yet Rated,NaN,57166,99999,반대가 끌리는 이유,NaN
110,96665ebeadb34eb08e026a5b5bcf5c81,Out of this World Girlfriend,2022,NaN,NaN,South Korea,Drama,2,720.0,2022-11-21,2022-11-23,"Monday, Wednesday",NaN,Not Yet Rated,A suspenseful comedy-drama about 3 unpopular k...,51931,99999,세상에 없는 여자친구,NaN
111,fee91be5beab46cc9e1a814f7feafc33,Summer Strike,2022,NaN,NaN,South Korea,Drama,12,3000.0,2022-11-21,2022-12-26,Monday,ENA Genie TV,15+ - Teens 15 or older,Lee Yeo Reum is a young woman whose career and...,818,494,아무것도 하고 싶지 않아,NaN
112,28ead17fd9c840459f25987f97212026,New Love Playlist,2022,['Kang Min Kyung'],['Choi Yo Ji'],South Korea,Drama,12,900.0,2022-11-18,2022-12-28,"Wednesday, Friday",Viki,13+ - Teens 13 or older,“New Love Playlist” will tell the story of col...,5067,4789,뉴연플리,NaN
113,3b079f4c99ae4b22bffc4f8cb5c9443e,"Fanletter, Please",2022,['Jung Sang Hee'],NaN,South Korea,Drama,4,3720.0,2022-11-18,2022-11-26,"Friday, Saturday",MBC,15+ - Teens 15 or older,Han Kang Hee is a top star actress who started...,1239,1414,팬레터를 보내주세요,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,d17381edcad24edd8b34d3a6466e00ee,The World of CC,2021,NaN,NaN,South Korea,Drama,8,600.0,2021-07-12,2021-08-29,"Monday, Sunday",NaN,Not Yet Rated,NaN,55744,99999,CC의 세계,NaN
435,2b15b1b97b264c85a56ab42854364276,Our First: Seventeen,2021,['Kim Soo Jung'],NaN,South Korea,Drama,10,600.0,2021-07-07,2021-08-07,"Wednesday, Saturday",NaN,15+ - Teens 15 or older,"A pure, harmless, no-MSG teen drama depicting ...",6283,7568,첫번째 열일곱,NaN
436,1171578051a44b5a93b8c691b6dd1d32,Red Shoes,2021,['Park Ki Hyun'],['Hwang Soon Young'],South Korea,Drama,100,2100.0,2021-07-05,2021-12-10,"Monday, Tuesday, Wednesday, Thursday, Friday",KBS2,15+ - Teens 15 or older,A story about a heartless mother who betrayed ...,8425,7601,빨강 구두,NaN
437,6cbb1fa1db7447379032df5d48e54e45,You Are My Spring,2021,"['Jung Ji Hyun', 'Joo Sang Kyu']",['Lee Mi Na'],South Korea,Drama,16,4200.0,2021-07-05,2021-08-24,"Monday, Tuesday",Netflix tvN,15+ - Teens 15 or older,When Kang Da Jung finds a job at a five-star h...,1884,630,너는 나의 봄,NaN


In [27]:
for i in range(109, 439):
    korean_drama.loc[i, 'genre'] = drama_genres[i-109]

In [30]:
korean_drama.loc[200]

kdrama_id                        b966838adf3547069389d6226ad55a67
title                                              Becoming Witch
year                                                         2022
director                                       ['Kim Yoon Cheol']
screenwriter                                      ['Park Pa Ran']
country                                               South Korea
type                                                        Drama
tot_eps                                                        12
duration                                                   4200.0
start_dt                                               2022-06-25
end_dt                                                 2022-09-10
aired_on                                                 Saturday
org_net                                                 TV Chosun
content_rt                                15+ - Teens 15 or older
synopsis        Three friends from different backgrounds each ...
rank      

In [32]:
korean_drama.loc[439]

kdrama_id       7d4e424bfb8a46078662c8ad89bbc1db
title                              Meeting Plaza
year                                        2021
director                                     NaN
screenwriter                                 NaN
country                              South Korea
type                                       Drama
tot_eps                                        8
duration                                   660.0
start_dt                              2021-07-01
end_dt                                2021-08-19
aired_on                                Thursday
org_net                                      NaN
content_rt               13+ - Teens 13 or older
synopsis                                     NaN
rank                                       99999
pop                                        99999
korean_title                              만남의 광장
genre                                        NaN
Name: 439, dtype: object

In [33]:
korean_drama.loc[439:776]['title']

439               Meeting Plaza
440            Re-Feel: If Only
441                 Light on Me
442            I'm Not a  Loser
443    On the Verge of Insanity
                 ...           
771             Winter Vacation
772                A Twist Girl
773       Dr. Romantic Season 2
774         Lover of the Palace
775                       Touch
Name: title, Length: 337, dtype: object

In [34]:
driver = webdriver.Chrome()

drama_eng_title = korean_drama.loc[439:776]['title']

def drama_genre(x):
    global drama_eng_title
    name = x
    url = f"https://mydramalist.com/search?q={name.replace(' ', '+')}&adv=titles&ty=68,83&so=relevance"
    driver.get(url)
    
    dramas = driver.find_element(By.XPATH, "//*[@id]/div/div/div[2]/h6/a[1]").send_keys(Keys.ENTER)
    try:
        drama_genre = driver.find_element(By.CLASS_NAME, "show-genres")
        drama_genre_text = drama_genre.text
    except Exception as ex:
        drama_genre_text = "Notfound"
    return drama_genre_text

drama_genres = []

i=0
# 영문 제목 번역 
for title in drama_eng_title:
    time.sleep(3)
    genre = drama_genre(title)
    drama_genres.append(genre)
    print(i, title, genre)
    i+=1
# 장르 결과
korean_drama['genre'] = drama_genres

driver.quit()

0 Meeting Plaza Genres: Comedy, Romance, Youth, Melodrama
1 Re-Feel: If Only Genres: Comedy, Romance, Fantasy
2 Light on Me Genres: Romance, Youth, Drama
3 I'm Not a  Loser Genres: Drama
4 On the Verge of Insanity Genres: Comedy, Life, Drama
5 Nevertheless, Genres: Romance, Life, Youth, Drama
6 Genesis Genres: Romance, Drama, Sci-Fi
7 Voice Season 4: Judgment Hour Genres: Action, Thriller, Mystery, Crime
8 So Not Worth it Genres: Comedy, Romance, Youth, Sitcom
9 Romanced Genres: Mystery, Romance
10 Hospital Playlist Season 2 Genres: Romance, Life, Drama, Medical
11 Yunseul Genres: Historical, Romance, Fantasy
12 On the Turf Genres: Romance
13 Monthly Magazine Home Genres: Business, Comedy, Romance, Drama
14 At a Distance, Spring Is Green Genres: Romance, Life, Youth
15 Love (ft. Marriage and Divorce) Season 2 Genres: Romance, Life, Drama
16 Starting Point of Dating Genres: Comedy, Romance, Youth
17 No Bother Me Genres: Romance, Youth
18 The Penthouse Season 3: War in Life Genres: Thril

TimeoutException: Message: timeout: Timed out receiving message from renderer: 299.536
  (Session info: chrome=116.0.5845.180)
Stacktrace:
	GetHandleVerifier [0x00007FF65D3A52A2+57122]
	(No symbol) [0x00007FF65D31EA92]
	(No symbol) [0x00007FF65D1EE3AB]
	(No symbol) [0x00007FF65D1DC01A]
	(No symbol) [0x00007FF65D1DBF63]
	(No symbol) [0x00007FF65D1DA9E5]
	(No symbol) [0x00007FF65D1DB31E]
	(No symbol) [0x00007FF65D1E80CB]
	(No symbol) [0x00007FF65D1F7F31]
	(No symbol) [0x00007FF65D1FC0DA]
	(No symbol) [0x00007FF65D1DB8E7]
	(No symbol) [0x00007FF65D1F7E0A]
	(No symbol) [0x00007FF65D25EBE9]
	(No symbol) [0x00007FF65D246DB3]
	(No symbol) [0x00007FF65D21D2B1]
	(No symbol) [0x00007FF65D21E494]
	GetHandleVerifier [0x00007FF65D64EF82+2849794]
	GetHandleVerifier [0x00007FF65D6A1D24+3189156]
	GetHandleVerifier [0x00007FF65D69ACAF+3160367]
	GetHandleVerifier [0x00007FF65D436D06+653702]
	(No symbol) [0x00007FF65D32A208]
	(No symbol) [0x00007FF65D3262C4]
	(No symbol) [0x00007FF65D3263F6]
	(No symbol)

In [35]:
drama_genres

['Genres: Comedy, Romance, Youth, Melodrama',
 'Genres: Comedy, Romance, Fantasy',
 'Genres: Romance, Youth, Drama',
 'Genres: Drama',
 'Genres: Comedy, Life, Drama',
 'Genres: Romance, Life, Youth, Drama',
 'Genres: Romance, Drama, Sci-Fi',
 'Genres: Action, Thriller, Mystery, Crime',
 'Genres: Comedy, Romance, Youth, Sitcom',
 'Genres: Mystery, Romance',
 'Genres: Romance, Life, Drama, Medical',
 'Genres: Historical, Romance, Fantasy',
 'Genres: Romance',
 'Genres: Business, Comedy, Romance, Drama',
 'Genres: Romance, Life, Youth',
 'Genres: Romance, Life, Drama',
 'Genres: Comedy, Romance, Youth',
 'Genres: Romance, Youth',
 'Genres: Thriller, Mystery, Crime, Drama',
 'Genres: Comedy, Romance, Drama',
 'Genres: Comedy, Romance, Supernatural',
 'Genres: Comedy, Life, Drama, Sports',
 'Genres: Romance',
 'Genres: Comedy, Drama, Melodrama',
 'Genres: Comedy, Romance, Fantasy',
 'Genres: Comedy, Romance, Drama',
 'Genres: Romance',
 'Genres: Romance, Drama',
 'Genres: Music',
 'Genres: 

In [36]:
korean_drama[korean_drama['title']=='To Me to You']

,kdrama_id,title,year,director,screenwriter,country,type,tot_eps,duration,start_dt,end_dt,aired_on,org_net,content_rt,synopsis,rank,pop,korean_title,genre
486,ee529948362243169e4f637f80633ad5,To Me to You,2021,NaN,NaN,South Korea,Drama,1,3600.0,"Apr 19, 2021","Apr 19, 2021",Monday,KBS1,15+ - Teens 15 or older,"Reflects on the meaning of a 'life friend', th...",46815,12506,나의 너에게,NaN


In [38]:
korean_drama.loc[486]

kdrama_id                        ee529948362243169e4f637f80633ad5
title                                                To Me to You
year                                                         2021
director                                                      NaN
screenwriter                                                  NaN
country                                               South Korea
type                                                        Drama
tot_eps                                                         1
duration                                                   3600.0
start_dt                                             Apr 19, 2021
end_dt                                               Apr 19, 2021
aired_on                                                   Monday
org_net                                                      KBS1
content_rt                                15+ - Teens 15 or older
synopsis        Reflects on the meaning of a 'life friend', th...
rank      

In [39]:
for i in range(439, 487):
    korean_drama.loc[i, 'genre'] = drama_genres[i-439]

In [42]:
korean_drama.loc[487]

kdrama_id       1455a0b6c8e344fb8d4f4cecf96ea60c
title                            School of Start
year                                        2021
director                                     NaN
screenwriter                                 NaN
country                              South Korea
type                                       Drama
tot_eps                                        3
duration                                   900.0
start_dt                              2021-04-16
end_dt                                2021-04-30
aired_on                                  Friday
org_net                                      NaN
content_rt               15+ - Teens 15 or older
synopsis                                     NaN
rank                                       53174
pop                                        99999
korean_title                          스쿨 오브 스타트업
genre                                        NaN
Name: 487, dtype: object

In [43]:
driver = webdriver.Chrome()

drama_eng_title = korean_drama.loc[487:776]['title']

def drama_genre(x):
    global drama_eng_title
    name = x
    url = f"https://mydramalist.com/search?q={name.replace(' ', '+')}&adv=titles&ty=68,83&so=relevance"
    driver.get(url)
    
    dramas = driver.find_element(By.XPATH, "//*[@id]/div/div/div[2]/h6/a[1]").send_keys(Keys.ENTER)
    try:
        drama_genre = driver.find_element(By.CLASS_NAME, "show-genres")
        drama_genre_text = drama_genre.text
    except Exception as e:
        drama_genre_text = "Notfound"
    return drama_genre_text

drama_genres = []

i=0
# 영문 제목 번역 
for title in drama_eng_title:
    time.sleep(3)
    genre = drama_genre(title)
    drama_genres.append(genre)
    print(i, title, genre)
    i+=1
# 장르 결과
korean_drama['genre'] = drama_genres

driver.quit()

0 School of Start Genres: Youth, Drama
1 Nice Na Ik Soo Genres: Comedy, Drama
2 Nobleman Ryu's Wedding Genres: Historical, Comedy, Romance, Drama
3 Sell Your Haunted House Genres: Thriller, Mystery, Comedy, Fantasy
4 Law School Genres: Mystery, Law, Crime, Drama
5 Translation-needed Love Season 2 Genres: Romance
6 My Love: Six Stories of True Love Genres: Drama, Documentary
7 Amor Fati Genres: Romance, Drama
8 Wanna Sleepover Genres: Romance, Drama
9 Taxi Driver Genres: Action, Thriller, Crime, Drama
10 Bio Homme Genres: Psychological, Life, Drama
11 Dramaworld Season 2 Genres: Comedy, Romance, Drama, Supernatural
12 Heart.zip Genres: Romance
13 Take It or Leave It Genres: Romance, Drama
14 Love Campus Genres: Comedy, Romance, Youth
15 Summer Guys Genres: Music, Romance, Youth, Drama
16 The Secret of the Grand Mansion: The Missing Girls Genres: Mystery, Comedy
17 Be My Dream Family Genres: Comedy, Romance, Drama, Family
18 Time to Be Together 1:11 Genres: Music, Romance, Youth
19 Scrip

ValueError: Length of values (289) does not match length of index (776)

In [45]:
len(drama_genres)

289

In [48]:
(korean_drama.loc[487:776])

,kdrama_id,title,year,director,screenwriter,country,type,tot_eps,duration,start_dt,end_dt,aired_on,org_net,content_rt,synopsis,rank,pop,korean_title,genre
487,1455a0b6c8e344fb8d4f4cecf96ea60c,School of Start,2021,NaN,NaN,South Korea,Drama,3,900.0,2021-04-16,2021-04-30,Friday,NaN,15+ - Teens 15 or older,NaN,53174,99999,스쿨 오브 스타트업,NaN
488,b51c154111534c679922411f32ca5270,Nice Na Ik Soo,2021,NaN,NaN,South Korea,Drama,5,180.0,"Apr 16, 2021","Apr 16, 2021",Friday,NaN,Not Yet Rated,NaN,45747,99999,나이스 나익수,NaN
489,ec8c0dd70a4b487280a12b64d8406d10,Nobleman Ryu's Wedding,2021,NaN,NaN,South Korea,Drama,8,660.0,2021-04-15,2021-05-06,Thursday,NaN,13+ - Teens 13 or older,On a warm spring day during the Joseon Dynast...,7844,729,류선비의 혼례식,NaN
490,ea1a5735be0a4e0baf90e79e9e24a1eb,Sell Your Haunted House,2021,['Park Jin Suk'],"['Ha Soo Jin', 'Jung Yeon Seo', 'Lee Young Hwa']",South Korea,Drama,16,4200.0,2021-04-14,2021-06-09,"Wednesday, Thursday",KBS2,15+ - Teens 15 or older,"Ji Ah, a woman who has inherited exorcism abil...",766,468,대박부동산,NaN
491,71a36358fd704b80b56042110f60a2f8,Law School,2021,['Kim Seok Yoon'],['Seo In'],South Korea,Drama,16,3900.0,2021-04-14,2021-06-09,"Wednesday, Thursday",jTBC Netflix,15+ - Teens 15 or older,When a grim incident occurs at their prestigio...,329,198,로스쿨,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,8a720bdcf2234bda87d53eb50736f699,Winter Vacation,2020,NaN,['Kim Hyo Jin'],South Korea,Drama,6,660.0,2020-01-10,2020-02-14,Friday,vLive,15+ - Teens 15 or older,It is a teen romance web drama that tells the ...,58926,13663,겨울방학,NaN
772,75b8778871df4fbc830652abeb13fa89,A Twist Girl,2020,NaN,NaN,South Korea,Drama,3,600.0,2020-01-09,2020-02-08,NaN,NaN,Not Yet Rated,The first-class girl in the whole school and a...,52785,99999,반전소녀,NaN
773,79c4732493654aea9f181811e289adc4,Dr. Romantic Season 2,2020,"['Yoo In Shik', 'Lee Gil Bok']",['Kang Eun Kyung'],South Korea,Drama,16,4200.0,2020-01-06,2020-02-25,"Monday, Tuesday",SBS,15+ - Teens 15 or older,"A “real doctor” story set in a small, humble h...",185,168,낭만닥터 김사부2,NaN
774,544c51af0c98459bb4e3ef922b56fffd,Lover of the Palace,2020,NaN,NaN,South Korea,Drama,10,600.0,2020-01-03,2020-01-17,NaN,NaN,Not Yet Rated,"Working as a tour guide at Deoksugung Palace, ...",8535,8135,궁궐의연인,NaN


In [49]:
for i in range(487, 776):
    korean_drama.loc[i, 'genre'] = drama_genres[i-487]

In [50]:
korean_drama

,kdrama_id,title,year,director,screenwriter,country,type,tot_eps,duration,start_dt,end_dt,aired_on,org_net,content_rt,synopsis,rank,pop,korean_title,genre
0,661d4193916c4e71a2c70473ab11e9e8,Sing My Crush,2023,['So Joon Moon'],NaN,South Korea,Drama,8,1500.0,"Aug 2, 2023","Aug 2, 2023",Wednesday,NaN,Not Yet Rated,Follow the story of acquaintances Ba Ram and H...,1484,2238,따라바람,"Genres: Music, Comedy, Romance, Youth"
1,5ffcbeaa17114714af1959129984274c,D.P. Season 2,2023,NaN,['Kim Bo Tong'],South Korea,Drama,6,3000.0,"Jul 28, 2023","Jul 28, 2023",Friday,Netflix,15+ - Teens 15 or older,This unfolding story ensues when military dese...,164,1084,디피 시즌2,"Genres: Action, Military, Drama"
2,65075cb9c1a54be4a441cee6f16c9fdf,Shadow Detective Season 2,2023,['Han Dong Hwa'],"['Song Jung Woo', 'Hwang Seol Hun']",South Korea,Drama,8,3300.0,2023-07-05,2023-07-26,Wednesday,Disney+ Hulu,15+ - Teens 15 or older,Unfolds the ultimate counterattack of veteran ...,2443,6915,형사록 시즌2,"Genres: Mystery, Drama"
3,df0f0ac4b3ff4b15afa26f5a7a53a328,To Be Honest,2023,NaN,NaN,South Korea,Drama,3,600.0,2023-06-30,2023-07-14,Friday,NaN,Not Yet Rated,Don't you have those days where the whole univ...,49895,99999,솔직하게 말해서,"Genres: Youth, Drama"
4,04c1fe41948e464fb440001831d74d41,Celebrity,2023,['Kim Chul Gyu'],['Kim Yi Young'],South Korea,Drama,12,2700.0,"Jun 30, 2023","Jun 30, 2023",Friday,Netflix,18+ Restricted (violence & profanity),Fame. Money. Power. One young woman fights to ...,826,547,셀러브리티,"Genres: Thriller, Mystery, Drama"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,8a720bdcf2234bda87d53eb50736f699,Winter Vacation,2020,NaN,['Kim Hyo Jin'],South Korea,Drama,6,660.0,2020-01-10,2020-02-14,Friday,vLive,15+ - Teens 15 or older,It is a teen romance web drama that tells the ...,58926,13663,겨울방학,"Genres: Romance, Youth"
772,75b8778871df4fbc830652abeb13fa89,A Twist Girl,2020,NaN,NaN,South Korea,Drama,3,600.0,2020-01-09,2020-02-08,NaN,NaN,Not Yet Rated,The first-class girl in the whole school and a...,52785,99999,반전소녀,"Genres: Youth, Supernatural"
773,79c4732493654aea9f181811e289adc4,Dr. Romantic Season 2,2020,"['Yoo In Shik', 'Lee Gil Bok']",['Kang Eun Kyung'],South Korea,Drama,16,4200.0,2020-01-06,2020-02-25,"Monday, Tuesday",SBS,15+ - Teens 15 or older,"A “real doctor” story set in a small, humble h...",185,168,낭만닥터 김사부2,"Genres: Romance, Drama, Medical, Melodrama"
774,544c51af0c98459bb4e3ef922b56fffd,Lover of the Palace,2020,NaN,NaN,South Korea,Drama,10,600.0,2020-01-03,2020-01-17,NaN,NaN,Not Yet Rated,"Working as a tour guide at Deoksugung Palace, ...",8535,8135,궁궐의연인,"Genres: Comedy, Romance"


In [2]:
korean_drama.to_csv('./korean_drama_genres.csv', index=False)

NameError: name 'korean_drama' is not defined

In [3]:
korean_drama = pd.read_csv('./korean_drama_genres.csv')

In [4]:
korean_drama

,kdrama_id,title,year,director,screenwriter,country,type,tot_eps,duration,start_dt,end_dt,aired_on,org_net,content_rt,synopsis,rank,pop,korean_title,genre
0,661d4193916c4e71a2c70473ab11e9e8,Sing My Crush,2023,['So Joon Moon'],NaN,South Korea,Drama,8,1500.0,"Aug 2, 2023","Aug 2, 2023",Wednesday,NaN,Not Yet Rated,Follow the story of acquaintances Ba Ram and H...,1484,2238,따라바람,"['Music', 'Comedy', 'Romance', 'Youth']"
1,5ffcbeaa17114714af1959129984274c,D.P. Season 2,2023,NaN,['Kim Bo Tong'],South Korea,Drama,6,3000.0,"Jul 28, 2023","Jul 28, 2023",Friday,Netflix,15+ - Teens 15 or older,This unfolding story ensues when military dese...,164,1084,디피 시즌2,"['Action', 'Military', 'Drama']"
2,65075cb9c1a54be4a441cee6f16c9fdf,Shadow Detective Season 2,2023,['Han Dong Hwa'],"['Song Jung Woo', 'Hwang Seol Hun']",South Korea,Drama,8,3300.0,2023-07-05,2023-07-26,Wednesday,Disney+ Hulu,15+ - Teens 15 or older,Unfolds the ultimate counterattack of veteran ...,2443,6915,형사록 시즌2,"['Mystery', 'Drama']"
3,df0f0ac4b3ff4b15afa26f5a7a53a328,To Be Honest,2023,NaN,NaN,South Korea,Drama,3,600.0,2023-06-30,2023-07-14,Friday,NaN,Not Yet Rated,Don't you have those days where the whole univ...,49895,99999,솔직하게 말해서,"['Youth', 'Drama']"
4,04c1fe41948e464fb440001831d74d41,Celebrity,2023,['Kim Chul Gyu'],['Kim Yi Young'],South Korea,Drama,12,2700.0,"Jun 30, 2023","Jun 30, 2023",Friday,Netflix,18+ Restricted (violence & profanity),Fame. Money. Power. One young woman fights to ...,826,547,셀러브리티,"['Thriller', 'Mystery', 'Drama']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,8a720bdcf2234bda87d53eb50736f699,Winter Vacation,2020,NaN,['Kim Hyo Jin'],South Korea,Drama,6,660.0,2020-01-10,2020-02-14,Friday,vLive,15+ - Teens 15 or older,It is a teen romance web drama that tells the ...,58926,13663,겨울방학,"['Romance', 'Youth']"
772,75b8778871df4fbc830652abeb13fa89,A Twist Girl,2020,NaN,NaN,South Korea,Drama,3,600.0,2020-01-09,2020-02-08,NaN,NaN,Not Yet Rated,The first-class girl in the whole school and a...,52785,99999,반전소녀,"['Youth', 'Supernatural']"
773,79c4732493654aea9f181811e289adc4,Dr. Romantic Season 2,2020,"['Yoo In Shik', 'Lee Gil Bok']",['Kang Eun Kyung'],South Korea,Drama,16,4200.0,2020-01-06,2020-02-25,"Monday, Tuesday",SBS,15+ - Teens 15 or older,"A “real doctor” story set in a small, humble h...",185,168,낭만닥터 김사부2,"['Romance', 'Drama', 'Medical', 'Melodrama']"
774,544c51af0c98459bb4e3ef922b56fffd,Lover of the Palace,2020,NaN,NaN,South Korea,Drama,10,600.0,2020-01-03,2020-01-17,NaN,NaN,Not Yet Rated,"Working as a tour guide at Deoksugung Palace, ...",8535,8135,궁궐의연인,"['Comedy', 'Romance']"


In [115]:
korean_drama.loc[109]

kdrama_id       c123bdee96414ea88041f11b0f046000
title                      Why Opposites Attract
year                                        2022
director                                     NaN
screenwriter                                 NaN
country                              South Korea
type                                       Drama
tot_eps                                        8
duration                                   480.0
start_dt                              2022-11-21
end_dt                                2023-01-02
aired_on                                     NaN
org_net                                      NaN
content_rt                         Not Yet Rated
synopsis                                     NaN
rank                                       57166
pop                                        99999
korean_title                          반대가 끌리는 이유
genre                                       ['']
Name: 109, dtype: object

In [122]:
korean_drama[korean_drama['genre']=="['']"]

,kdrama_id,title,year,director,screenwriter,country,type,tot_eps,duration,start_dt,end_dt,aired_on,org_net,content_rt,synopsis,rank,pop,korean_title,genre
109,c123bdee96414ea88041f11b0f046000,Why Opposites Attract,2022,NaN,NaN,South Korea,Drama,8,480.0,2022-11-21,2023-01-02,NaN,NaN,Not Yet Rated,NaN,57166,99999,반대가 끌리는 이유,['']
239,cc8355c5316f444e98a24cb2f83e6801,The Escape Story of a Plastic Spoon,2022,NaN,NaN,South Korea,Drama,2,240.0,2022-04-25,2022-05-01,NaN,NaN,Not Yet Rated,NaN,63185,99999,흙수저 탈출기,['']
309,67cc937e9a6245e6ac31e0ffeb8f2442,Love & Wish,2021,NaN,['Han Yoon Sun'],South Korea,Drama,9,960.0,2021-12-24,2021-12-25,"Friday, Saturday",Daum Kakao TV,15+ - Teens 15 or older,"Son Da Eun, 17, is the youngest of three siste...",4525,2650,사랑,['']
724,bf51403a87d94534a75a352814dd73c9,ShorTest,2020,NaN,NaN,South Korea,Drama,2,360.0,2020-04-10,2020-04-17,Friday,NaN,Not Yet Rated,NaN,99999,99999,ShorTest,['']


In [123]:
korean_drama = pd.read_csv('./korean_drama_genres.csv')

In [124]:
korean_drama

,kdrama_id,title,year,director,screenwriter,country,type,tot_eps,duration,start_dt,end_dt,aired_on,org_net,content_rt,synopsis,rank,pop,korean_title,genre
0,661d4193916c4e71a2c70473ab11e9e8,Sing My Crush,2023,['So Joon Moon'],NaN,South Korea,Drama,8,1500.0,"Aug 2, 2023","Aug 2, 2023",Wednesday,NaN,Not Yet Rated,Follow the story of acquaintances Ba Ram and H...,1484,2238,따라바람,"['Music', 'Comedy', 'Romance', 'Youth']"
1,5ffcbeaa17114714af1959129984274c,D.P. Season 2,2023,NaN,['Kim Bo Tong'],South Korea,Drama,6,3000.0,"Jul 28, 2023","Jul 28, 2023",Friday,Netflix,15+ - Teens 15 or older,This unfolding story ensues when military dese...,164,1084,디피 시즌2,"['Action', 'Military', 'Drama']"
2,65075cb9c1a54be4a441cee6f16c9fdf,Shadow Detective Season 2,2023,['Han Dong Hwa'],"['Song Jung Woo', 'Hwang Seol Hun']",South Korea,Drama,8,3300.0,2023-07-05,2023-07-26,Wednesday,Disney+ Hulu,15+ - Teens 15 or older,Unfolds the ultimate counterattack of veteran ...,2443,6915,형사록 시즌2,"['Mystery', 'Drama']"
3,df0f0ac4b3ff4b15afa26f5a7a53a328,To Be Honest,2023,NaN,NaN,South Korea,Drama,3,600.0,2023-06-30,2023-07-14,Friday,NaN,Not Yet Rated,Don't you have those days where the whole univ...,49895,99999,솔직하게 말해서,"['Youth', 'Drama']"
4,04c1fe41948e464fb440001831d74d41,Celebrity,2023,['Kim Chul Gyu'],['Kim Yi Young'],South Korea,Drama,12,2700.0,"Jun 30, 2023","Jun 30, 2023",Friday,Netflix,18+ Restricted (violence & profanity),Fame. Money. Power. One young woman fights to ...,826,547,셀러브리티,"['Thriller', 'Mystery', 'Drama']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,8a720bdcf2234bda87d53eb50736f699,Winter Vacation,2020,NaN,['Kim Hyo Jin'],South Korea,Drama,6,660.0,2020-01-10,2020-02-14,Friday,vLive,15+ - Teens 15 or older,It is a teen romance web drama that tells the ...,58926,13663,겨울방학,"['Romance', 'Youth']"
772,75b8778871df4fbc830652abeb13fa89,A Twist Girl,2020,NaN,NaN,South Korea,Drama,3,600.0,2020-01-09,2020-02-08,NaN,NaN,Not Yet Rated,The first-class girl in the whole school and a...,52785,99999,반전소녀,"['Youth', 'Supernatural']"
773,79c4732493654aea9f181811e289adc4,Dr. Romantic Season 2,2020,"['Yoo In Shik', 'Lee Gil Bok']",['Kang Eun Kyung'],South Korea,Drama,16,4200.0,2020-01-06,2020-02-25,"Monday, Tuesday",SBS,15+ - Teens 15 or older,"A “real doctor” story set in a small, humble h...",185,168,낭만닥터 김사부2,"['Romance', 'Drama', 'Medical', 'Melodrama']"
774,544c51af0c98459bb4e3ef922b56fffd,Lover of the Palace,2020,NaN,NaN,South Korea,Drama,10,600.0,2020-01-03,2020-01-17,NaN,NaN,Not Yet Rated,"Working as a tour guide at Deoksugung Palace, ...",8535,8135,궁궐의연인,"['Comedy', 'Romance']"


In [5]:
origin_drama_info = pd.read_csv('./origin_drama_info.csv')

In [6]:
origin_drama_info

,Unnamed: 0,drama_name,org_net,director,screenwriter,start_dt,end_dt,aired_on,year,tot_eps,synopsis,actor_list
0,0,Sing My Crush,NaN,['So Joon Moon'],NaN,"Aug 2, 2023","Aug 2, 2023",Wednesday,2023,8,Follow the story of acquaintances Ba Ram and H...,"['Jang Do Yoon', 'Son Hyun Woo']"
1,1,D.P. Season 2,Netflix,NaN,['Kim Bo Tong'],"Jul 28, 2023","Jul 28, 2023",Friday,2023,6,This unfolding story ensues when military dese...,"['Jung Hae In', 'Koo Kyo Hwan', 'Kim Sung Kyun..."
2,2,Shadow Detective Season 2,Disney+ Hulu,['Han Dong Hwa'],"['Song Jung Woo', 'Hwang Seol Hun']",2023-07-05,2023-07-26,Wednesday,2023,8,Unfolds the ultimate counterattack of veteran ...,"['Lee Sung Min', 'Jung Jin Young', 'Kyung Soo ..."
3,3,To Be Honest,NaN,NaN,NaN,2023-06-30,2023-07-14,Friday,2023,3,Don't you have those days where the whole univ...,['Baek Hyo Won']
4,4,Celebrity,Netflix,['Kim Chul Gyu'],['Kim Yi Young'],"Jun 30, 2023","Jun 30, 2023",Friday,2023,12,Fame. Money. Power. One young woman fights to ...,"['Park Gyu Young', 'Kang Min Hyuk', 'Lee Chung..."
...,...,...,...,...,...,...,...,...,...,...,...,...
772,772,Winter Vacation,vLive,NaN,['Kim Hyo Jin'],2020-01-10,2020-02-14,Friday,2020,6,It is a teen romance web drama that tells the ...,"['Seo Sung Hyuk', 'Lee Hwi Seo', 'Cho Yeong In']"
773,773,A Twist Girl,NaN,NaN,NaN,2020-01-09,2020-02-08,NaN,2020,3,The first-class girl in the whole school and a...,"['Jung Lael', 'Kang Min Gyu']"
774,774,Dr. Romantic Season 2,SBS,"['Yoo In Shik', 'Lee Gil Bok']",['Kang Eun Kyung'],2020-01-06,2020-02-25,"Monday, Tuesday",2020,16,"A “real doctor” story set in a small, humble h...","['Han Seok Kyu', 'Lee Sung Kyung', 'Ahn Hyo Se..."
775,775,Lover of the Palace,NaN,NaN,NaN,2020-01-03,2020-01-17,NaN,2020,10,"Working as a tour guide at Deoksugung Palace, ...","['Han Bo Reum', 'Shin Won Ho']"
